# Overfitting and Regularization with Weight Decay

## Learning from Data: Homework 6

The following questions from this homework will be answered and its solutions will be explained: questions 2, 3, 4, 5 and 6

### Exercise

The following problems use [this training set](http://work.caltech.edu/data/in.dta) and [this test set](http://work.caltech.edu/data/out.dta).

Each line of the files corresponds to a two-dimensional input $x = (x_1,x_2)$, so that $\mathcal{X} = \mathbb{R}^2$, followed by the corresponding label from $\mathcal{Y} = \{-1, 1\}$. We are going to apply Linear Regression with a non-linear transformation for classification. The nonlinear transformation is given by

$$\phi(x_1, x_2) = (1, x_1, x_2, x_1^2, x_2^2, x_1x_2, |x_1 - x_2|, |x_1 + x_2|)$$


Recall that the classification error is defined as the fraction of missclassified points.

In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_table("in.dta", sep=" +", header=None, engine='python')
train.columns = ["x1", "x2", "y"]
test = pd.read_table("out.dta", sep=" +", header=None, engine='python')
test.columns = ["x1", "x2", "y"]

In [4]:
for df in [train, test]:
    df['one'] = 1
    df['x1^2'] = df['x1']**2
    df['x2^2'] = df['x2']**2
    df['x1x2'] = df['x1']*df['x2']
    df['|x1-x2|'] = np.abs(df['x1'] - df['x2'])
    df['|x1+x2|'] = np.abs(df['x1'] + df['x2'])

In [5]:
train = train.reindex(columns = ['one', 'x1', 'x2', 'x1^2', 'x2^2', 'x1x2', '|x1-x2|', '|x1+x2|', 'y'])
test = test.reindex(columns = ['one', 'x1', 'x2', 'x1^2', 'x2^2', 'x1x2', '|x1-x2|', '|x1+x2|', 'y'])

In [10]:
train.head()

,one,x1,x2,x1^2,x2^2,x1x2,|x1-x2|,|x1+x2|,y
0,1,-0.779470,0.838221,0.607574,0.702615,-0.653369,1.617692,0.058751,1
1,1,0.155635,0.895377,0.024222,0.801701,0.139352,0.739743,1.051012,1
2,1,-0.059908,-0.717780,0.003589,0.515208,0.043001,0.657872,0.777688,1
3,1,0.207596,0.758933,0.043096,0.575980,0.157552,0.551337,0.966530,1
4,1,-0.195983,-0.375487,0.038409,0.140991,0.073589,0.179504,0.571470,-1


First, we explain Linear Regression and non-linear transformations:

### Linear Regression

Linear Regression is a linear model that applies to real-values target functions. So, we have a data set $\mathcal{D}$ of training examples $(x_1, y_1), (x_2, y_2), \ldots, (x_n, y_n)$ where $x_n$ are the input values and $y_n$ are the real-valued output values or targets. Our goal is to find a hypothesis $g$ that minimizes the amount of misclassified points.

Since our targets are now in the real domain, they will no longer be a deterministic function $y=f(x)$. Instead, it will be a noisy target function formalized as a distribution of the random variable $y$. More precisely, the label $y_n$ comes from some distribution $P(y|x)$. So, we have an unknown distribution $P(x,y)$ that generates our training examples $(x_n, y_n)$ and we want to find a hypothesis $g$ that minimizes the error between $g(x)$ and $y$ w.r.t. that distribution. The assumption made here is that there exists a linear combination of the input values $x$ that properly approximates the output values $y$. If this assumption does not hold, we can opt for a non-linear transformation, which will be discussed later on.

The linear regression algorithm tries to minimize the out-of-sample error, $E_{out}(h)$, by minimizing the squared error between the hypothesis $h(x)$ and the output $y$

$$E_{out}(h) = \mathbb{E} \left[(h(x)-y)^2 \right]$$

where the expected value is taken w.r.t. the probability distribution $P(x,y)$. Since the distribution $P(x,y)$ is unknown, $E_{out}(h)$ can not be computed. So, we must use the in-sample error instead:

$$E_{in}(h) = \frac{1}{N} \sum_{n=1}^{N}{\left( h(x_n) - y_n \right)^2}$$

As was already mentioned, in linear regression, the hypothesis $h$ takes the form of a linear combination of the components of the inputs $x$. Formally:

$$h(x) = \sum_{i=0}^{d}{w_i x_i} = w^Tx$$

where $x_0 = 1$. For linear regression, it is very useful to represent the inputs and outputs as matrices. The data matrix $X \in \mathbb{R}^{N \times (d+1)}$ is the $N \times (d+1)$ matrix whose rows are the inputs $x_n$ as row vectors. The target vector $y \in \mathbb{R}^N$ is the column vector whose components are the target values $y_n$. This can be summarized as:

$$X=\left[ \begin{matrix} x_{ 1 }^{ T } \\ x_{ 2 }^{ T } \\ \vdots  \\ x_N^T \end{matrix} \right], \qquad y=\left[ \begin{matrix} y_1 \\ y_2 \\ \vdots  \\ y_N \end{matrix} \right] $$

With this representation, the linear regression algorithm can be derived and consists of the following steps:

 1. Construct the matrix $X$ and $y$ as explained above.
 2. Compute the pseudo-inverse $X^{\dagger}$ of the matrix $X$. If $X^TX$ is invertible, $$X^{\dagger} = (X^TX)^{-1}X^T$$
 3. Return $w_{lin} = X^{\dagger}y$.
 
This is exactly what the first code fragment below does. The function **lin_regression** computes the dot product of the pseudo-inverse of $X$ and $y$. The returned value is assigned to the weight vector $w$. The second function, **class_error**, computes the classification error by comparing the outputs found with our learned weight vector $w$ to the real outputs.

### Non-linear Transformation

As mentioned, linear regression assumes a linear combination of the inputs $x$ to properly approximate the outputs $y$. This assumption is clear when looking at the form of the hypothesis $h$:

$$h(x) = w^Tx$$

This quantity is not only linear in terms of the $x_i$'s, but also in terms of the $w_i$'s. In fact, the linearity of the $w_i$'s is the only thing that matters, since these need to be learned. The $x_i$'s are merely constants. This observation allows us to apply non-linear transformations to the $x_i$'s, while still using the linear model as stated above, since the resulting model remains linear in terms of the $w_i$'s. Applying non-linear transformations to the inputs $x_i$ allows for more rich information contained within these inputs.

A non-linear transformation on the inputs $x$ is usually denoted by the function $\phi(x)$. This results in a vector $z$. While the inputs $x$ live in the $\mathcal{X}$ space, the non-linearly transformed inputs $z$ live in the $\mathcal{Z}$ space. The $\mathcal{Z}$ space is also referred to as the feature space since it contains higher-level features derived from the raw inputs $x$. The transformation function $\phi$ is called the feature transform.

With these non-linear transformations, we can now represent a non-linear hypothesis $h$ in the $\mathcal{X}$ space by a linear hypothesis $\tilde{h}$ in the $\mathcal{Z}$ space. More formally:

$$h(x) = \tilde{h}(\phi(x))$$

As was already mentioned in the exercise, the non-linear transformation $\phi$ applied to the inputs $x$ is:

$$\phi(x_1, x_2) = (1, x_1, x_2, x_1^2, x_2^2, x_1x_2, |x_1 - x_2|, |x_1 + x_2|)$$

The code responsible for this is the 3th code block of this Notebook.

In [11]:
def lin_regression(Xs, Ys):
    # Dot product of pseudo-inverse of X and Y
    return np.dot(np.linalg.pinv(Xs), Ys)

# One step learning: determine the weights
w = lin_regression(train.ix[:,:-1], train['y'])

In [13]:
# Classification error: Compute outputs with current w
# and compare to real outputs
def class_error(Xs, Ys, w):
    Y_est = np.sign(np.dot(Xs, w))
    return np.mean(Y_est != Ys)

### Question 2

Run Linear Regression on the training set after performing the non-linear transformation and compute the in-sample and out-of-sample error.

In [7]:
# Classification error on training set
class_error(train.ix[:,:-1], train['y'], w)

0.028571428571428571

In [8]:
# Classification error on test set
class_error(test.ix[:,:-1], test['y'], w)

0.084000000000000005

It can be seen that these values are the closest to the one of answer A ($0.03$ and $0.08$).

Both the in-sample and out-of-sample error are already small, indicating a good performance of the Linear Regression algorithm. It might be possible to improve this performance even further. Maybe this model slightly overfits the data? One way to combat overfitting is by using Regularized Linear Regression with Weight Decay. What these concepts are will be explained in the following section.

### Regularized Linear Regression with Weight Decay

We can say a model overfits the data when it is able to produce very accuracte predictions on the training data, i.e. the seen data, but it does not generalize well to unseen data. In fact, the model is trained on the training data so much that is has learned to fit the noise in that data. So, one can notice overfitting has occured when picking a hypothesis $g$ with a lower value for $E_{in}$ and getting a higher value for $E_{out}$.

One way to deal with overfitting is to use Regularization. This technique will constraint the learning algorithm to improve the out-of-sample error. One way to write this down formally is as follows:

$$E_{out}(h) \le E_{in}(h) + \Omega(\mathcal{H}), \qquad \text{for all}~h \in \mathcal{H}$$

where $\Omega(\mathcal{H})$ is a model complexity penalty. This tells us we will be better off by fitting the data using a simple hypothesis $h \in \mathcal{H}$. The difficulty of Regularization is finding a suitable complecity measure $\Omega(\mathcal{H})$. With Regularization, one not only minimizes $E_{in}$, but the combination of $E_{in}$ and $\Omega(\mathcal{H})$.

One way of Regularization is called Weight Decay. First, we define the augmented error as:

$$E_{aug}(w) = E_{in}(w) + \lambda w^T w$$

where $\lambda \ge 0$ is a free parameter. This augmented error has two terms: the in-sample error and the penalty term discussed earlier. The aim of Weight Decay is to minimize this penalized in-sample error. The value of $\lambda$ controls the amount of regularization, while the penalty term $w^T w$ enforces a tradeoff between making the in-sample error small and making the weights small. With a technique like gradient descent e.g., we find a reducation in in-sample error together with a gradual shrinking of the weights. In fact, a more general definition of the augmented error can be given:

$$E_{aug}(h, \lambda, \Omega) = E_{in}(h) + \frac{\lambda}{N} \Omega(h)$$

This defition is more suitable, since we are not only in control of the hypothesis $h$, but also over the amount of regularization $\lambda$ and the penalty term $\Omega(h)$, which we chose to be $w^T w$ for weight decay. The need for regularization goes down as the data set size $N$ increases, so $\frac{1}{N}$ was left out in the previous equation. This allows for $\lambda$ to be less sensitive to $N$.

As was the case with the normal variant of Linear Regression described above, so is also Regularized Linear Regression a one-step-learning algorithm. It can be derived that for Regularized Linear Regression with Weight Decay the weight vector $w_{reg}$ can be calculated as follows:

$$w_{reg} = (X^T X + \lambda I)^{-1} X^T y$$

In the code fragments below, we did not directly specify a value for the parameter $\lambda$. Instead, we defined $\lambda$ as being of the following form:

$$\lambda = 10^{k}$$

and provided the parameter $k$.

In [9]:
# Regularized linear regression
def regul_lin_regression(Xs, Ys, k):
    l = 10**k
    A = np.linalg.inv(np.dot(Xs.T, Xs) + np.eye(Xs.shape[1])*l)
    B = np.dot(Xs.T, Ys)
    return np.dot(A,B)

# Weights with k = -3, i.e. lambda = 10E-3
w_neg_k = regul_lin_regression(train.ix[:,:-1], train['y'], -3)

### Question 3

Add Weight Decay to the Linear Regression and compute the in-sample and out-of-sample error for $k=-3$.

In [10]:
#Classification error on train set
#with k = -3, i.e. lambda = 10E-3
class_error(train.ix[:,:-1], train['y'], w_neg_k)

0.028571428571428571

In [11]:
#Classification error on test set
#with k = -3, i.e. lambda = 10E-3
class_error(test.ix[:,:-1], test['y'], w_neg_k)

0.080000000000000002

The values for the in- and out-of sample error are closest to the values of answer D ($0.03$ and $0.08$).

As can be seen from these results, using a small amount of regularization ($\lambda = 0.001$) did not improve the performance of the algorithm. The in-sample and out-of-sample errors are still te same. Maybe we should try more regularization.

### Question 4

Add Weight Decay to the Linear Regression and compute the in-sample and out-of-sample error for $k=3$.

In [12]:
# Weights with k = 3, i.e. lambda = 10E3
w_pos_k = regul_lin_regression(train.ix[:,:-1], train['y'], 3)

In [13]:
#Classification error on train set
#with k = 3, i.e. lambda = 10E3
class_error(train.ix[:,:-1], train['y'], w_pos_k)

0.37142857142857144

In [14]:
#Classification error on test set
#with k = 3, i.e. lambda = 10E3
class_error(test.ix[:,:-1], test['y'], w_pos_k)

0.436

We see that the in- and out-of sample errors when using $k=3$ are closest to the ones of answer E ($0.4$ and $0.4$).

As can be seen from these results, applying too much regularization ($\lambda = 1000$) worsens the algorithm's performance. In the following section, a good value for $k$ will be searched.

### Question 5

Add Weight Decay to the Linear Regression and see which value of $k$ produces the smalles in-sample and out-of-sample error for $k=[2,1,0,-1,-2]$.

In [15]:
# Find minimum E_out
def min_E_out(ks):
    errors = np.zeros(len(ks))
    for i,k in enumerate(ks):
        w_est = regul_lin_regression(train.ix[:, :-1], train['y'], k)
        error = class_error(test.ix[:,:-1], test['y'], w_est)
        errors[i] = error
    min_k = np.argmin(errors)
    return (errors, errors[min_k], ks[min_k])

In [16]:
# Vary k over [2,1,0,-1,-2]
min_E_out([2,1,0,-1,2])

(array([ 0.228,  0.124,  0.092,  0.056,  0.228]), 0.056000000000000001, -1)

The smallest out-of-sample classification error is achieved when we use $k=-1$, which is answer D.

From these results, it can be seen that the minimal out-of-sample error ($E_{out} = 0.056$) is found when $k=-1$, or $\lambda = 0.1$. So the ideal amount of regularization was in-between what we tried earlier.

### Question 6

Add Weight Decay to the Linear Regression and see which value of $k$ produces the smalles in-sample and out-of-sample error, limiting $k$ to integer values.

In [17]:
# Vary k over range [-10, 10]
min_E_out(np.arange(-10,11))

(array([ 0.084,  0.084,  0.084,  0.084,  0.084,  0.084,  0.084,  0.08 ,
         0.084,  0.056,  0.092,  0.124,  0.228,  0.436,  0.452,  0.456,
         0.456,  0.456,  0.456,  0.456,  0.456]), 0.056000000000000001, -1)

The value closest to the minimum out-of-sample classification error achieved by varying $k$ (limiting $k$ to integer values) is $k=0.06$, which is answer B.

Again, we see the optimal value for $\lambda$ is $\lambda = 0.1$.